In [30]:
print("hello")

hello


In [31]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [32]:
#Extract Data From the Pdf File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents

In [33]:
cd ..

c:\Users\Vaibhav\OneDrive\Desktop\gen_ai


c:\Users\Vaibhav\OneDrive\Desktop\gen_ai\End_to_End_Medical_chatbot\medibot\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [34]:
extracted_data=load_pdf_file(data='Data/')

FileNotFoundError: Directory not found: 'Data/'

In [28]:
# extracted_data

In [16]:
def split_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

In [ ]:
text_chunks= text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
#download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

C:\Users\Vaibhav\AppData\Local\Temp\ipykernel_17468\3963075778.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Vaibhav\OneDrive\Desktop\gen_ai\End_to_End_Medical_chatbot\medibot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
!pip install faiss-cpu
from langchain.vectorstores import FAISS




In [25]:
# Vectorize and store using FAISS
def create_vector_store(text_chunks):
    vectorstore = FAISS.from_documents(text_chunks, embeddings)
    return vectorstore

In [ ]:
data_path = "Data/"
documents = load_pdf_file(data_path)
text_chunks = split_documents(documents)
vectorstore = create_vector_store(text_chunks)

In [28]:

# Embedding using Hugging Face
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

c:\Users\Vaibhav\OneDrive\Desktop\gen_ai\End_to_End_Medical_chatbot\medibot\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vaibhav\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [35]:
query = "What are the symptoms of flu?"
retrieved_docs = vectorstore.similarity_search(query)

In [36]:
for doc in retrieved_docs:
    print(doc.page_content)  # Display the relevant documents

• painful, bluish or purplish fingers or toes
• puffy or swollen eyelids, face, feet, or lower legs
• changes in the color of the face
• skin rash,itching, or hives
• yellow eyes or skin
• severe or continuing headache
• sore throat and fever, with or without chills
• breathing problems or wheezing
• tightness in the chest
• dizziness
• unusual tiredness or weakness
• weight gain
In addition, patients taking anticoagulant drugs
should check with their physicians as soon as possible if
not function properly.
Symptoms
The symptoms of the above disorders include:
• Systemic lupus erythematosus. Symptoms include
fever, chills, fatigue, weight loss, skin rashes (particu-
larly the classic “butterfly” rash on the face), vasculitis,
polyarthralgia, patchy hair loss, sores in the mouth or
nose, lymph-node enlargement, gastric problems, and,
in women, irregular periods. About half of those who
suffer from lupus develop cardiopulmonary problems,
also be transmitted sexually.
The disease is not u

In [37]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

In [41]:
from dotenv import load_dotenv
import os

load_dotenv()
huggingface_token = os.getenv('HUGGINGFACE_API_TOKEN')

In [46]:
from dotenv import load_dotenv
import os

load_dotenv()
huggingface_token = os.getenv('HUGGINGFACE_API_TOKEN')

# Initialize HuggingFace Pipeline
model_id = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=huggingface_token)
model = AutoModelForCausalLM.from_pretrained(model_id, use_auth_token=huggingface_token)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=500,
    temperature=0.4,
    top_p=0.95,
    repetition_penalty=1.15
)

llm = HuggingFacePipeline(pipeline=pipe)


c:\Users\Vaibhav\OneDrive\Desktop\gen_ai\End_to_End_Medical_chatbot\medibot\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vaibhav\.cache\huggingface\hub\models--facebook--opt-350m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu
C:\Users\Vaibhav\AppData\Local\Temp\ipykernel_17468

In [47]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you don't know. "
    "Use 4 sentences maximum to keep the answer concise.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectorstore.as_retriever()
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Test the chain
response = rag_chain.invoke({"input": "What is correlation?"})
print(response["answer"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\Vaibhav\OneDrive\Desktop\gen_ai\End_to_End_Medical_chatbot\medibot\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Vaibhav\OneDrive\Desktop\gen_ai\End_to_End_Medical_chatbot\medibot\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based

System: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use 4 sentences maximum to keep the answer concise.

• Repression. The person pushes anxious thoughts or
ideas out of conscious awareness.
• Displacement. Anxiety from one source is attached to a
different object or event. Phobias are an example of the
mechanism of displacement in psychoanalytic theory.
• Rationalization. The person justifies the anxious feel-
ings by saying that any normal person would feel anx-
ious in their situation.
• Somatization. The anxiety emerges in the form of phys-
ical complaints and illnesses, such as recurrent

thinking, confusion, and loss of muscular coordi-
nation.
Antibody —A specific protein produced by the
immune system in response to a specific foreign
protein or particle called an antigen.
Antigen—Any substance that stimulates the body to
produce antibody.
Autoimmun